In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras import layers
import tensorflow_hub as hub
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences

NUM_WORDS = 8000
SEQ_LEN = 25

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

dataframe = pd.read_csv('amazon_pc_user_reviews.csv')
print(dataframe.head())

reviews = dataframe['review_body']
labels = dataframe['star_rating']

review_train, review_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.25, random_state=1000)

print(len(review_train), 'train examples')
print(len(review_test), 'test examples')

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(review_train.apply(lambda x: np.str_(x)))

x_train =  tokenizer.texts_to_sequences(review_train.apply(lambda x: np.str_(x)))
x_test = tokenizer.texts_to_sequences(review_test.apply(lambda x: np.str_(x)))

vocab_size = len(tokenizer.word_index) + 1
print(review_train[2])
print(x_train[2])

maxlen = 256

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)
print(x_train[0, :])

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)
print("Accuracy:", score)

embedding_dim = 50

model = keras.Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=10, validation_data=(x_test, y_test) ,epochs=20, verbose=1)

loss, accuracy = model.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))


model.save('model.h5')

import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
plot_history(history)


Version:  2.0.0-beta1
Eager mode:  True
Hub version:  0.6.0
GPU is NOT AVAILABLE
                                         review_body  star_rating
0                                     very satisfied            5
1  installation was easy and the cover provides t...            4
2  this pcie adapter by rosewill is outstanding m...            5
3  product is pretty cheap in price and is of dec...            4
4  when my tower died overnight i had to decide o...            4
75000 train examples
25000 test examples
this pcie adapter by rosewill is outstanding my router is on the first floor of my home and my pc is on the third even under heavy broadband loads on my road and area this adapter still pushes out  mbs on average which is beyond fast for home internet in columbia sc max speeds i have hit  mbs which like i said is amazing for home internet i have reviewed the asus ac as well and for the price this card smokes the asus if you are willing to pay the money the asus is the better bu

C:\Users\wardp\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\wardp\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\wardp\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.58528
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 256, 50)           3141100   
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 3,141,621
Trainable params: 3,141,621
Non-trainable params: 0
_________________________________________________________________
Train on 75000 samples, validate on 25000 samples
Epoch 1/20
61270/75000 [=======================>......] - ETA: 6s - loss: -46.9909 - accuracy: 0.1104